<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 3: Future projections for each impact

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 4 included the
manuscript <<< insert doi >>>. Future risks for fire. We show median changes in
risk with respect to the historic.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
from showit import image
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import regionmask as rm
import altair as alt
alt.themes.enable("carbonplan_light")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4

In [ ]:
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)  # this right? or just on/off

In [ ]:
ds = (
    xr.open_zarr(get_store("carbonplan-scratch", "data/fire_future_v3.zarr"))
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(mask)
    .groupby("time.year")
    .sum()
    .where(mask)
    .compute()
)

In [ ]:
ds_remaining = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_future_v3_remaining.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(mask)
    .compute()
)

In [ ]:
ds = xr.merge([ds, ds_remaining])

In [ ]:
historical_annual = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(mask)
    .mean(dim=["x", "y"])
    .compute()
)

In [ ]:
future_maps = ds.sel(year=slice("2070", "2099")).mean(dim="year").compute()
future_ts = ds.mean(dim=["x", "y"]).compute()

## Plot the figure


In [ ]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
scenario_colors = {
    "ssp245": "#7EB36A",
    "ssp370": "#D4C05E",
    "ssp585": "#F07071",
}
titles = [
    "Burn area\n[fraction/year]",
    "Drought mortality\n[]",
    "Biotic agent mortality\n[]",
]

In [ ]:
fig, axarr = plt.subplots(
    ncols=3, nrows=6, figsize=(6, 10), sharey=True, sharex=True
)
for (gcm, location) in gcms:
    historical_annual["historical"].plot(
        ax=axarr[location], color="k", linestyle="--", label="Historical"
    )

    future_ts["{}_{}".format(gcm, "ssp245")].sel(
        year=slice("1971", "2015")
    ).plot(ax=axarr[location], color="k", alpha=0.5)
    future_ts["{}_{}".format(gcm, "ssp245")].sel(
        year=slice("1971", "2015")
    ).rolling(dim={"year": 10}).mean().plot(
        ax=axarr[location], color="k", label="Control"
    )

    for scenario in [
        "ssp585",
        "ssp245",
        "ssp370",
    ]:
        try:
            future_ts["{}_{}".format(gcm, scenario)].sel(
                year=slice("2015", "2099")
            ).plot(
                ax=axarr[location], color=scenario_colors[scenario], alpha=0.5
            )
            future_ts["{}_{}".format(gcm, scenario)].sel(
                year=slice("2015", "2099")
            ).rolling(dim={"year": 10}).mean().plot(
                ax=axarr[location],
                color=scenario_colors[scenario],
                label=scenario,
            )
        except:
            print("oops!")
    axarr[location].set_xlabel("")
    axarr[location].set_yticks([0, 0.01, 0.02, 0.03])
    #     axarr[location].set_yscale('log')
    axarr[location].set_ylim(0, 0.03)
    if location[1] == 0:
        axarr[location].set_ylabel(gcm)

for i, title in enumerate(titles):
    axarr[0, i].title.set_text(title)
for ax in axarr.flatten():
    for side in ["top", "bottom", "right", "left"]:
        ax.spines[side].set_visible(False)
plt.tight_layout()
plt.legend()
plt.savefig("futuretraces.png", dpi=300)

In [ ]:
ds_rolling = future_ts.rolling(dim={"year": 10}).mean()

In [ ]:
fig, axarr = plt.subplots(ncols=3, figsize=(10, 4), sharey=True)
historical_annual["historical"].plot(
    ax=axarr[0], color="k", linestyle="--", label="Historical"
)
for scenario in ["ssp245", "ssp370", "ssp585"]:
    for (gcm, location) in gcms:
        ds_rolling["{}_{}".format(gcm, scenario)].sel(
            year=slice("2015", "2099")
        ).plot(ax=axarr[0], color=scenario_colors[scenario], alpha=0.2)

    ds_rolling[["{}_{}".format(gcm[0], scenario) for gcm in gcms]].to_array(
        dim="vars"
    ).mean("vars").plot(
        ax=axarr[0], color=scenario_colors[scenario], label=scenario
    )
axarr[0].set_xlabel("")
axarr[0].set_ylim(0, 0.015)
axarr[0].set_ylim(0, 0.015)

axarr[0].legend()

for i, impact in enumerate(titles):
    axarr[i].set_ylabel(impact)
impacts = ["Fire", "Drought", "Biotic Agents"]
for i, impact in enumerate(impacts):
    axarr[i].set_title(impact)

In [ ]:
map_2090s = future_maps.to_array(dim="vars")  # .mean('vars')

In [ ]:
panels = map_2090s.plot(x="x", y="y", col="vars", col_wrap=3, vmin=0, vmax=0.2)